In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras as kr
import utils

In [4]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [5]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#Load the data
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [7]:
x_train = train.drop(["label"], axis=1).astype("float32")
y_train = train["label"].astype("int32")
x_test = test.astype("float32")
x_train = (x_train.values.reshape(-1, 28, 28, 1))/255.0
x_test = (x_test.values.reshape(-1, 28, 28, 1))/255.0
y_train = kr.utils.to_categorical(y_train, 10)
x_train.shape, x_test.shape, y_train.shape

((42000, 28, 28, 1), (28000, 28, 28, 1), (42000, 10))

In [8]:
model = kr.models.Sequential([
    kr.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    kr.layers.Conv2D(32, (3, 3), activation="relu"),
    kr.layers.MaxPooling2D(2, 2),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    kr.layers.Dropout(0.25),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.Conv2D(64, (3, 3), activation="relu", padding="Same"),
    kr.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    kr.layers.Dropout(0.25),
    kr.layers.Flatten(),
    kr.layers.Dense(256, activation="relu"),
    kr.layers.Dense(256, activation="relu"),
    kr.layers.Dropout(0.50),
    kr.layers.Dense(10, activation="softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 64)          0

In [10]:
y_train.shape, model.output_shape

((42000, 10), (None, 10))

In [27]:
optimizer = kr.optimizers.Adam(
    learning_rate = 0.0005,
    beta_1= 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = "Adam"
)

In [28]:
class myCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        
    def on_epoch_end(self, epoch, logs):
        if(logs.get("accuracy") > 0.999):
            self.model.stop_training= True
            print("\nReached 99.9% of accuracy so cancelling training")

In [29]:
callbacks = myCallback()
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

In [32]:
model.fit(x = x_train, y = y_train, batch_size=50, epochs = 1)

840/840 [==============================] - 132s 158ms/step - loss: 0.0275 - accuracy: 0.9919


In [33]:
results = model.predict(x_test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")

In [37]:
image_id = pd.Series(range(1, 28001), name = "ImageId")
submission = pd.concat([image_id, results], axis = 1)
submission.to_csv("../output/submission.csv", index=False)